Importando dados e libs

In [1]:
import keras
import tensorflow as tf

In [2]:
with open("txt\definicao_ptbr_extracao.txt", "r", encoding="utf-8") as file:
    text = file.read()

In [3]:
text = text.replace('*', '')

In [4]:
# Tokenizado o texto

tokenizer = keras.layers.TextVectorization(
    standardize="lower",
    split="character",
    output_mode="int",
)

tokenizer.adapt([text])

In [5]:
len(tokenizer.get_vocabulary()) - 2

57

In [6]:
token = tokenizer([text])[0]
token = token - 2

In [7]:
dataset = tf.data.Dataset.from_tensor_slices(token)

In [8]:
janela = 50

In [9]:
dataset = dataset.window(janela, shift=1, drop_remainder=True)
dataset = dataset.flat_map(lambda window: window.batch(janela))

In [10]:
dataset = dataset.shuffle(1000).batch(20)
dataset = dataset.map(lambda window:(window[:, :-1], window[:, 1:]))

In [11]:
dataset = dataset.prefetch(1)

Criando modelo

In [12]:
tam = len(tokenizer.get_vocabulary()) - 2
tam

57

In [ ]:
model = keras.models.Sequential([
    keras.layers.Embedding(
        input_dim=tam,
        output_dim=80,
    ),
    keras.layers.GRU(200, return_sequences=True),
    keras.layers.GRU(200, return_sequences=True, dropout=0.2, recurrent_dropout=0.1),
    keras.layers.TimeDistributed(
        keras.layers.Dense(
            tam,
            activation=keras.activations.softmax,
            kernel_initializer=keras.initializers.glorot_uniform
        )
    )
])
model.compile(
    loss=keras.losses.SparseCategoricalCrossentropy(),
    optimizer=keras.optimizers.Adam(),
    metrics=["accuracy"]
)
model.fit(
    dataset,
    epochs=20,
    callbacks=[
        keras.callbacks.EarlyStopping(
            patience=4,
            restore_best_weights=True
        ),
    ],
    )
model.save(filepath='model_llm.keras' )

Epoch 1/20
195/195 ━━━━━━━━━━━━━━━━━━━━ 67s 271ms/step - accuracy: 0.3207 - loss: 2.4155
Epoch 2/20


c:\Users\mateu\Documents\Norton\Projetos GIT\trust-advisor-streamlit-finance\venv\Lib\site-packages\keras\src\trainers\epoch_iterator.py:164: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()
c:\Users\mateu\Documents\Norton\Projetos GIT\trust-advisor-streamlit-finance\venv\Lib\site-packages\keras\src\callbacks\early_stopping.py:99: UserWarning: Early stopping conditioned on metric `val_loss` which is not available. Available metrics are: accuracy,loss
  current = self.get_monitor_value(logs)


195/195 ━━━━━━━━━━━━━━━━━━━━ 49s 251ms/step - accuracy: 0.6480 - loss: 1.2122
Epoch 3/20
195/195 ━━━━━━━━━━━━━━━━━━━━ 52s 265ms/step - accuracy: 0.8407 - loss: 0.5754
Epoch 4/20
195/195 ━━━━━━━━━━━━━━━━━━━━ 54s 274ms/step - accuracy: 0.9096 - loss: 0.3384
Epoch 5/20
195/195 ━━━━━━━━━━━━━━━━━━━━ 52s 265ms/step - accuracy: 0.9286 - loss: 0.2589
Epoch 6/20
195/195 ━━━━━━━━━━━━━━━━━━━━ 54s 275ms/step - accuracy: 0.9355 - loss: 0.2236
Epoch 7/20
195/195 ━━━━━━━━━━━━━━━━━━━━ 54s 275ms/step - accuracy: 0.9399 - loss: 0.2031
Epoch 8/20
195/195 ━━━━━━━━━━━━━━━━━━━━ 51s 259ms/step - accuracy: 0.9428 - loss: 0.1900
Epoch 9/20
195/195 ━━━━━━━━━━━━━━━━━━━━ 47s 240ms/step - accuracy: 0.9449 - loss: 0.1804
Epoch 10/20
195/195 ━━━━━━━━━━━━━━━━━━━━ 47s 238ms/step - accuracy: 0.9464 - loss: 0.1738
Epoch 11/20
195/195 ━━━━━━━━━━━━━━━━━━━━ 47s 241ms/step - accuracy: 0.9475 - loss: 0.1679
Epoch 12/20
195/195 ━━━━━━━━━━━━━━━━━━━━ 45s 231ms/step - accuracy: 0.9488 - loss: 0.1629
Epoch 13/20
195/195 ━━━━━━━━━

In [14]:
pred = model.predict(tokenizer(['o que é retorno acumulado?']))[0, -1]
y_pred = tf.argmax(pred)  # choose the most probable character ID
tokenizer.get_vocabulary()[y_pred + 1]

1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step


'[UNK]'

In [15]:
model_final = keras.models.Sequential([
    keras.layers.Lambda(lambda X: X - 2),
    model
])

In [16]:
def next_char(text, temperature=1.0):
    encoder_text = tokenizer([text])

    y_proba = model_final.predict(encoder_text, verbose=0)[0, -1:]
    
    rescaled_logits = tf.math.log(y_proba) / temperature
    y_pred = tf.random.categorical(rescaled_logits, num_samples=1)[0, 0]
    return tokenizer.get_vocabulary()[y_pred + 2]

In [17]:
def generate_text(text, n_chars=1000, temperature=1.0):
    for _ in range(n_chars):
        str_ = next_char(text, temperature)
        if str_ == '.':
            break
        text += str_
    return text

In [18]:
generate_text("o que é retorno acumulado",n_chars=100, temperature=0.8)

'o que é retorno acumulado\nrepresenta o crescimento total do capital investido ao longo de um período, considerando a capitali'